In [1]:
import sys
sys.path.append('/Users/brandon/Documents/Repositories/Python/')
from FANC_auto_recon.annotations import schema_download
from FANC_auto_recon import neuroglancer_utilities
import pandas as pd
import numpy as np
from nglui import statebuilder,annotation,easyviewer,parser

In [2]:
client,token = neuroglancer_utilities.get_client()

In [3]:
neuroglancer_utilities.get_cv_path()

{'Image': {'url': 'precomputed://gs://zetta_lee_fly_vnc_001_precomputed/vnc1_full_v3align_2/realigned_v1',
  'resolution': '[4.3,4.3,45]'},
 'Flat_1': {'url': 'https://storage.googleapis.com/zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v1',
  'resolution': '[4.3,4.3,45]'},
 'FlatV_2': {'url': 'https://storage.googleapis.com/zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v2',
  'resolution': '[4.3,4.3,45]'},
 'Dynamic_V1': {'url': 'graphene://https://wclee.api.zetta.ai/segmentation/table/vnc1_full_v3align_2',
  'resolution': '[17.2,17.2,45]'},
 'Dynamic_V2': {'url': 'graphene://https://wclee.api.zetta.ai/segmentation/table/vnc1_full_v3align_2_v2',
  'resolution': '[17.2,17.2,45]'},
 'Flat_3': {'url': 'precomputed://gs://zetta_lee_fly_vnc_001_segmentation/vnc1_full_v3align_2/realigned_v1/seg/full_run_v3',
  'resolution': '[4.3,4.3,45]'},
 'Dynamic_V4': {'url': 'graphene://https://wclee.api.zetta.ai/segmentation/table/v

In [4]:
def find_neurons(tag, client=None, segmentation_version='Dynamic_V4', return_IDs = True, partial_match = True):
    if client is None:
        client,token = neuroglancer_utilities.get_client()
        
    tables = client.annotation.get_tables()
    annotations = pd.DataFrame(columns=['deleted', 'valid', 'schema_type', 'reference_table', 'user_id',
       'created', 'table_name', 'id', 'flat_segmentation_source',
       'description', 'tag', 'pt_position', 'superceded_id'])
    for i in tables:
        meta = client.annotation.get_table_metadata(i)
        if meta['schema_type'] == 'bound_tag':
            try:
                return(schema_download.download_annotation_table(client,i))
                annotations = annotations.append(schema_download.download_annotation_table(client,i))
            except:
                print(i +' Failed')
    
    if partial_match is True:
        queried_annotations = annotations.loc[[tag in i for i in annotations.tag]]
    else:
        queried_annotations = annotations.loc[annotations.tag == tag]
        
    if len(queried_annotations) > 0:
        materialized_annotations = schema_download.generate_soma_table(queried_annotations,segmentation_version=segmentation_version)
    else:
        return('No neurons matching this query')
    
    materialized_annotations['table_name'] = queried_annotations.table_name
    if return_IDs:
        return(materialized_annotations.pt_root_id.values)

    return(materialized_annotations)

In [5]:
find_neurons('MN_L112',return_IDs=False)

,deleted,valid,schema_type,reference_table,user_id,created,table_name,id,flat_segmentation_source,description,tag,pt_position,superceded_id
0,None,True,NaN,NaN,NaN,2021-01-15 00:03:33.264701,test_T1MN_soma_table,1,NaN,NaN,MN_A101_T1L,"[13329, 114635, 1861]",None
1,None,True,NaN,NaN,NaN,2021-01-15 00:03:33.582501,test_T1MN_soma_table,2,NaN,NaN,MN_A101_T1R,"[63394, 114393, 2304]",None
2,None,True,NaN,NaN,NaN,2021-01-15 00:03:33.720227,test_T1MN_soma_table,3,NaN,NaN,MN_A102_T1L,"[11729, 116735, 1478]",None
3,None,True,NaN,NaN,NaN,2021-01-15 00:03:33.850803,test_T1MN_soma_table,4,NaN,NaN,MN_A102_T1R,"[63084, 116459, 2863]",None
4,None,True,NaN,NaN,NaN,2021-01-15 00:03:33.981768,test_T1MN_soma_table,5,NaN,NaN,MN_A103_T1R,"[59659, 115855, 2065]",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,None,True,NaN,NaN,NaN,2021-01-15 00:03:48.924138,test_T1MN_soma_table,117,NaN,NaN,MN_V402_T1L,"[23167, 87333, 2184]",None
56,None,True,NaN,NaN,NaN,2021-01-15 00:03:49.057628,test_T1MN_soma_table,118,NaN,NaN,MN_V501_T1L,"[27494, 86790, 2515]",None
57,None,True,NaN,NaN,NaN,2021-01-15 00:03:49.191420,test_T1MN_soma_table,119,NaN,NaN,MN_V501_T1R,"[54883, 84397, 2319]",None
58,None,True,NaN,NaN,NaN,2021-01-15 00:03:49.324049,test_T1MN_soma_table,120,NaN,NaN,MN_V601_T1R,"[49727, 85190, 2264]",None


In [6]:
client.annotation.get_tables()

['test_synapse_table',
 'test_synapse_table_2',
 'test_T1MN_soma_table',
 'T1MN_somas',
 '10B_somas',
 'claw_neurons',
 '81A07_Synapses',
 'test_table',
 'club_synapses',
 'claw_neuron_synapses',
 'test',
 'synapses',
 'MB_synapses',
 'Ltm synapse',
 'Mollie_synapses',
 'claw_neuron_synapses_2',
 'Hemilineages',
 '20210225 Mollie_synapses',
 'claw_neuron_synapses_3',
 'Ltm synapse-2-3',
 'tiny Ltm synapse-4-1',
 'contr-ltm-midbranch-4-1',
 'cellbody-ltm-midbranch-4-2',
 'AIZ-ltm-midbranch-4-5',
 'cellbody-ltm-midbranch-4-5',
 'tiny-ltm-4-5',
 'contra-contraltm-4-5']

In [7]:
client.annotation.get_table_metadata('test_T1MN_soma_table')

{'deleted': '2021-01-15T08:15:57.356269',
 'valid': True,
 'schema_type': 'bound_tag',
 'reference_table': None,
 'user_id': 'bmark89@uw.edu',
 'created': '2021-01-15T00:03:32.919454',
 'table_name': 'test_T1MN_soma_table',
 'id': 4,
 'flat_segmentation_source': None,
 'description': 'test table for bound_tag schema'}

In [ ]:
test.loc[['ee' in i for i in test.coords]]

In [ ]:
img_info = neuroglancer_utilities.get_cv_path('Image')
img_path = img_info['url']
img_res = img_info['resolution']
img_layer = statebuilder.ImageLayerConfig(img_path,name = 'Image')

seg_info_dynamic = neuroglancer_utilities.get_cv_path('Dynamic_V4')
seg_info_flat = neuroglancer_utilities.get_cv_path(version = 'Flat_3')

In [ ]:
def render_flat(img_path,seg_path,ids=None,resolution = [4.3,4.3,45]):
    img_layer = statebuilder.ImageLayerConfig(img_path,name = 'Image')
    seg_layer = statebuilder.SegmentationLayerConfig(seg_path,name='Segmentation',fixed_ids=ids)
    sb = statebuilder.StateBuilder(layers=[img_layer, seg_layer],resolution=resolution)
    return(sb.render_state(return_as='html', link_text='Flat Segmentation'))



    

In [ ]:
render_flat(img_path,seg_info_flat['url'])

In [ ]:
state = client.state.get_state_json()

In [ ]:
nglui.parser